In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np
import pandas as pd
import keras
import os
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import img_to_array
print(img_to_array)
from tensorflow.keras.utils import load_img
print(load_img)
from keras.applications.vgg19 import VGG19 , preprocess_input , decode_predictions

2022-12-04 16:31:18.093810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 16:31:18.276409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 16:31:18.276457: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-04 16:31:19.552594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

<function img_to_array at 0x7f5ff637ab90>
<function load_img at 0x7f5ff637acb0>


In [3]:
train_datagen = ImageDataGenerator(zoom_range=0.5, shear_range=0.3, horizontal_flip=True, preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
train = train_datagen.flow_from_directory(directory= '/workspaces/test/datasets/data_distribution_for_SVM/train' , target_size=(256, 256), batch_size=32)
val = val_datagen.flow_from_directory(directory= '/workspaces/test/datasets/data_distribution_for_SVM/test' , target_size=(256, 256), batch_size=32)

In [9]:
train, val = get_dataset_partitions_tf(dataset)


In [10]:
def show_img(img_array , label):
    for im , l in zip(img_array , label):
     plt.figure(figsize=(5, 5))
    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.imshow(img[i])
        plt.title(label[i])
        plt.axis('off')
    plt.show()

In [ ]:
plot_img = (t_img[:3] , t_label[:3])

# model

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D , Flatten
import keras

In [ ]:
Base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
for layer in Base_model.layers:
    layer.trainable = False

In [ ]:
Base_model.summary()

In [ ]:
m = Flatten()(Base_model.output)
m = Dense(38, activation='softmax')(m)

In [ ]:
models = Model(Base_model.input, m)

In [ ]:
models.summary()

In [ ]:
models.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=3 , min_delta=0.01)
mc = ModelCheckpoint(filepath='best_model2.h5',
 monitor='val_accuracy',
   min_delta=0.01 ,
    verbose=1 ,
     save_best_only=True ,
     patience=3)
cb = [es, mc]

In [ ]:
his = models.fit_generator(train,
 validation_data=val,
 validation_steps=16,
 steps_per_epoch=16, 
 epochs=50,
 verbose=1,
 callbacks=cb)

In [ ]:
h = his.history
h.keys()


In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c = "red")
plt.title('acc vs val_acc')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] , c = "red")
plt.title('loss vs val_loss')
plt.show()

In [ ]:
from keras.models import load_model
Model = load_model('/workspaces/test/best_model.h5')
acc = Model.evaluate_generator(val)[1]
print(f"the accuracy of our model is {acc*100} %")

In [ ]:
ref = dict (zip (list (train.class_indices.values()) , list (train.class_indices.keys())))

In [ ]:
def prediction(path):
    img = load_img(path, target_size=(256, 256))
    i = img_to_array(img)
    im = preprocess_input(i)
    img = np.expand_dims(im, axis=0)
    pred = np.argmax(Model.predict(img))
    print(f"the prediction of the image is {ref[pred]}")

In [ ]:
path = "/workspaces/test/potato.jpeg"
prediction(path)